In [82]:
import requests
import pandas as pd
import numpy as np
import itertools as it
import urllib.request 
import json
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.io.json import json_normalize
from IPython.display import IFrame

from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
#!pip install jupyterthemes
#!jt -t chesterish
#http://data.nba.net/10s/prod/v1/2018/players.json
#https://www.techradar.com/how-to/computing/apple/terminal-101-creating-cron-jobs-1305651
#https://www.quora.com/Is-there-an-NBA-API-for-free-that-has-live-stats
#http://data.nba.net/10s/prod/v1/today.json

# Seasons for any player and team wins category

In [83]:
#Get Season Years into a DataFrame - from Russ Westbrook player profile data 
with urllib.request.urlopen("http://data.nba.net/10s/prod/v1/2018/players/201566_profile.json") as url:
    data = json.loads(url.read().decode())
seasonyears = data['league']['standard']['stats']['regularSeason']
seasonyears = json_normalize(seasonyears, 'season')
seasonyears = seasonyears['seasonYear']
dfseason = pd.DataFrame(data=seasonyears)

#Teamdata for the wins category section
urlteam = urllib.request.urlopen("http://data.nba.net/10s/prod/v1/current/standings_all_no_sort_keys.json")
response = json.loads(urlteam.read().decode())
dfteam = response['league']['standard']
dfteam = json_normalize(dfteam, 'teams')
dfteam = pd.DataFrame(dfteam)
dfteam = dfteam[['teamId','win']]
dfteam

,teamId,win
0,1610612761,23
1,1610612749,20
2,1610612743,20
3,1610612744,21
4,1610612754,20
5,1610612760,19
6,1610612755,20
7,1610612738,18
8,1610612747,18
9,1610612746,17


# Ham's gut guys

In [84]:
#Dictionary of Ham's players so that the for loop can input the data into player_data_Ham dictionary 
#for all player stats

TeamHam = {
    "AHorford": "http://data.nba.net/10s/prod/v1/2018/players/201143_profile.json",
    "BBeal": "http://data.nba.net/10s/prod/v1/2018/players/203078_profile.json",
    "KAT": "http://data.nba.net/10s/prod/v1/2018/players/1626157_profile.json",
    "KLove": "http://data.nba.net/10s/prod/v1/2018/players/201567_profile.json",
    "LAldridge": "http://data.nba.net/10s/prod/v1/2018/players/200746_profile.json",
    "DDerozan": "http://data.nba.net/10s/prod/v1/2018/players/201942_profile.json",
    "KD": "http://data.nba.net/10s/prod/v1/2018/players/201142_profile.json",
    "KIrving": "http://data.nba.net/10s/prod/v1/2018/players/202681_profile.json",
    "RWestbrook": "http://data.nba.net/10s/prod/v1/2018/players/201566_profile.json",
    "DLillard": "http://data.nba.net/10s/prod/v1/2018/players/203081_profile.json",
    "SCurry": "http://data.nba.net/10s/prod/v1/2018/players/201939_profile.json",
    "BGriffin": "http://data.nba.net/10s/prod/v1/2018/players/201933_profile.json"
    
    }

#Create empty Dictionary for Ham's players, player_data_Ham
player_data_Ham = {}

#loop through the values ie the .json links and pull the stats for seasonYear, ppg,rpg,bpg,apg,spg and into into the
#empty dictionary of player_data_Ham
for keyham, val in TeamHam.items():
    url = urllib.request.urlopen(val)
    response = json.loads(url.read().decode())
    df = response['league']['standard']['stats']['regularSeason']['season']
    df = json_normalize(df, 'teams')
    df = pd.DataFrame(df)
    df = pd.concat([dfseason, df], axis=1)
    df = df[['seasonYear','ppg','rpg','bpg','apg','spg','teamId']]
    df['name'] = keyham
    player_data_Ham[keyham] = df

player_data_Ham

{'AHorford':     seasonYear   ppg   rpg  bpg  apg  spg      teamId      name
 0       2018.0  12.4   6.4  1.7  3.4  0.8  1610612738  AHorford
 1       2017.0  12.9   7.4  1.1  4.7  0.6  1610612738  AHorford
 2       2016.0    14   6.8  1.3    5  0.8  1610612738  AHorford
 3       2015.0  15.2   7.3  1.5  3.2  0.8  1610612737  AHorford
 4       2014.0  15.2   7.2  1.3  3.2  0.9  1610612737  AHorford
 5       2013.0  18.6   8.4  1.5  2.6  0.9  1610612737  AHorford
 6       2012.0  17.4  10.2    1  3.2    1  1610612737  AHorford
 7       2011.0  12.4     7  1.3  2.2  0.9  1610612737  AHorford
 8       2010.0  15.3   9.3    1  3.5  0.8  1610612737  AHorford
 9       2009.0  14.2   9.9  1.1  2.3  0.7  1610612737  AHorford
 10      2008.0  11.5   9.3  1.4  2.4  0.8  1610612737  AHorford
 11         NaN  10.1   9.7  0.9  1.5  0.7  1610612737  AHorford,
 'BBeal':     seasonYear   ppg  rpg  bpg  apg  spg      teamId   name
 0         2018  23.2  4.6  0.9  4.4    1  1610612764  BBeal
 1         

In [85]:
#for keyham, val in TeamHam.items():
#    dfH1 = player_data_Ham[keyham]
#    dfH1 = pd.DataFrame(dfH1)
#    #dfH1 = dfH1.iloc[[0]]
#    df['name'] = keyham
#    dfH1[keyham] = dfH1 
#    
#dfH1

In [86]:
#Get all the players into their own dataframe and sellect only the last row for season 2018


dfH1 = player_data_Ham['AHorford']
dfH1 = pd.DataFrame(dfH1)
dfH1 = dfH1.iloc[[0]]

dfH2 = player_data_Ham['BBeal']
dfH2 = pd.DataFrame(dfH2)
dfH2 = dfH2.iloc[[0]]

dfH3 = player_data_Ham['KAT']
dfH3 = pd.DataFrame(dfH3)
dfH3 = dfH3.iloc[[0]]

dfH4 = player_data_Ham['KLove']
dfH4 = pd.DataFrame(dfH4)
dfH4 = dfH4.iloc[[0]]

dfH5 = player_data_Ham['LAldridge']
dfH5 = pd.DataFrame(dfH5)
dfH5 = dfH5.iloc[[0]]

dfH6 = player_data_Ham['DDerozan']
dfH6 = pd.DataFrame(dfH6)
dfH6 = dfH6.iloc[[0]]

dfH7 = player_data_Ham['KD']
dfH7 = pd.DataFrame(dfH7)
dfH7 = dfH7.iloc[[0]]

dfH8 = player_data_Ham['KIrving']
dfH8 = pd.DataFrame(dfH8)
dfH8 = dfH8.iloc[[0]]

dfH9 = player_data_Ham['RWestbrook']
dfH9 = pd.DataFrame(dfH9)
dfH9 = dfH9.iloc[[0]]

dfH10 = player_data_Ham['DLillard']
dfH10 = pd.DataFrame(dfH10)
dfH10 = dfH10.iloc[[0]]

dfH11 = player_data_Ham['SCurry']
dfH11 = pd.DataFrame(dfH11)
dfH11 = dfH11.iloc[[0]]

dfH12 = player_data_Ham['BGriffin']
dfH12 = pd.DataFrame(dfH12)
dfH12 = dfH12.iloc[[0]]

#Make all the player Dataframes into one team dataframe
dfTEAMHam = [dfH12, dfH11, dfH10, dfH9, dfH8, dfH7, dfH6, dfH5, dfH4, dfH3, dfH2, dfH1]
dfTEAMHam = pd.concat(dfTEAMHam)

#Get all the wins data and merge on teamId from two
dfTEAMHam = dfTEAMHam.merge(dfteam,on='teamId')


#The data is in str format, convert the 5 columns to float so that they can be sum'ed for the team data
dfTEAMHam['ppg'] = dfTEAMHam['ppg'].astype(float)
dfTEAMHam['rpg'] = dfTEAMHam['rpg'].astype(float)
dfTEAMHam['bpg'] = dfTEAMHam['bpg'].astype(float)
dfTEAMHam['apg'] = dfTEAMHam['apg'].astype(float)
dfTEAMHam['spg'] = dfTEAMHam['spg'].astype(float)
dfTEAMHam['win'] = dfTEAMHam['win'].astype(float)


#dfTEAMHam.dtypes

#Makes a new column in the final row as a team total
dfTEAMHam.loc['Total']= dfTEAMHam.sum()
dfTEAMHam

#Change seasonYear to just say 2018, and same with Team name
dfTEAMHam.iloc[-1, dfTEAMHam.columns.get_loc('seasonYear')] = 2018
dfTEAMHam.iloc[-1, dfTEAMHam.columns.get_loc('name')] = 'TeamHam'
dfTEAMHam = dfTEAMHam[['name','ppg', 'rpg','bpg','apg','spg','win']]


dfTEAMHam.tail()


,name,ppg,rpg,bpg,apg,spg,win
8,LAldridge,18.9,9.6,1.2,2.1,0.4,16.0
9,KLove,19.0,13.5,0.2,3.5,0.2,7.0
10,KAT,21.4,11.9,1.8,2.5,0.9,14.0
11,BBeal,23.2,4.6,0.9,4.4,1.0,12.0
Total,TeamHam,272.1,93.9,9.0,61.4,12.1,193.0


# Draymond Kickboxing Dojo

In [87]:
#Dictionary of Ham's players so that the for loop can input the data into player_data_Ham dictionary 
#for all player stats
TeamRic = {
    "CPaul": "http://data.nba.net/10s/prod/v1/2018/players/101108_profile.json",
    "JWall": "http://data.nba.net/10s/prod/v1/2018/players/202322_profile.json",
    "Kemba": "http://data.nba.net/10s/prod/v1/2018/players/202689_profile.json",
    "Bledsoe": "http://data.nba.net/10s/prod/v1/2018/players/202339_profile.json",
    "Kawhi": "http://data.nba.net/10s/prod/v1/2018/players/202695_profile.json",
    "LBJ": "http://data.nba.net/10s/prod/v1/2018/players/2544_profile.json",
    "PaulGeorge": "http://data.nba.net/10s/prod/v1/2018/players/202331_profile.json",
    "Giannis": "http://data.nba.net/10s/prod/v1/2018/players/203507_profile.json",
    "MarcGasol": "http://data.nba.net/10s/prod/v1/2018/players/201188_profile.json",
    "DAyton": "http://data.nba.net/10s/prod/v1/2018/players/1629028_profile.json",
    "Booker": "http://data.nba.net/10s/prod/v1/2018/players/1626164_profile.json",
    "Rubio": "http://data.nba.net/10s/prod/v1/2018/players/201937_profile.json"  
    }

#Create empty Dictionary for Ham's players, player_data_Ham
player_data_Ric = {}

#loop through the values ie the .json links and pull the stats for seasonYear, ppg,rpg,bpg,apg,spg and into into the
#empty dictionary of player_data_Ham

for key, val in TeamRic.items():
    url = urllib.request.urlopen(val)
    response = json.loads(url.read().decode())
    df = response['league']['standard']['stats']['regularSeason']['season']
    df = json_normalize(df, 'teams')
    df = pd.DataFrame(df)
    df = pd.concat([dfseason, df], axis=1)
    df = df[['seasonYear','ppg','rpg','bpg','apg','spg','teamId']]
    df['name'] = key
    player_data_Ric[key] = df

In [88]:
dfR1 = player_data_Ric['CPaul']
dfR1 = pd.DataFrame(dfR1)
dfR1 = dfR1.iloc[[0]]

dfR2 = player_data_Ric['JWall']
dfR2 = pd.DataFrame(dfR2)
dfR2 = dfR2.iloc[[0]]

dfR3 = player_data_Ric['Kemba']
dfR3 = pd.DataFrame(dfR3)
dfR3 = dfR3.iloc[[0]]

dfR4 = player_data_Ric['Bledsoe']
dfR4 = pd.DataFrame(dfR4)
dfR4 = dfR4.iloc[[0]]

dfR5 = player_data_Ric['Kawhi']
dfR5 = pd.DataFrame(dfR5)
dfR5 = dfR5.iloc[[0]]

dfR6 = player_data_Ric['LBJ']
dfR6 = pd.DataFrame(dfR6)
dfR6 = dfR6.iloc[[0]]

dfR7 = player_data_Ric['PaulGeorge']
dfR7 = pd.DataFrame(dfR7)
dfR7 = dfR7.iloc[[0]]

dfR8 = player_data_Ric['Giannis']
dfR8 = pd.DataFrame(dfR8)
dfR8 = dfR8.iloc[[0]]

dfR9 = player_data_Ric['MarcGasol']
dfR9 = pd.DataFrame(dfR9)
dfR9 = dfR9.iloc[[0]]

dfR10 = player_data_Ric['DAyton']
dfR10 = pd.DataFrame(dfR10)
dfR10 = dfR10.iloc[[0]]

dfR11 = player_data_Ric['Booker']
dfR11 = pd.DataFrame(dfR11)
dfR11 = dfR11.iloc[[0]]

dfR12 = player_data_Ric['Rubio']
dfR12 = pd.DataFrame(dfR12)
dfR12 = dfR12.iloc[[0]]

dfTEAMRic = [dfR12, dfR11, dfR10, dfR9, dfR8, dfR7, dfR6, dfR5, dfR4, dfR3, dfR2, dfR1]
dfTEAMRic = pd.concat(dfTEAMRic)

dfTEAMRic = dfTEAMRic.merge(dfteam,on='teamId')


dfTEAMRic['ppg'] = dfTEAMRic['ppg'].astype(float)
dfTEAMRic['rpg'] = dfTEAMRic['rpg'].astype(float)
dfTEAMRic['bpg'] = dfTEAMRic['bpg'].astype(float)
dfTEAMRic['apg'] = dfTEAMRic['apg'].astype(float)
dfTEAMRic['spg'] = dfTEAMRic['spg'].astype(float)
dfTEAMRic['win'] = dfTEAMRic['win'].astype(float)


#dfTEAMRic.dtypes


dfTEAMRic.loc['Total']= dfTEAMRic.sum()
dfTEAMRic

dfTEAMRic.iloc[-1, dfTEAMRic.columns.get_loc('seasonYear')] = 2018
dfTEAMRic.iloc[-1, dfTEAMRic.columns.get_loc('name')] = 'TeamRic'
dfTEAMRic = dfTEAMRic[['name','ppg', 'rpg','bpg','apg','spg','win']]


dfTEAMRic.tail()




,name,ppg,rpg,bpg,apg,spg,win
8,Kawhi,26.3,8.4,0.6,3.1,1.8,23.0
9,Kemba,24.9,4.5,0.4,6.3,1.3,14.0
10,JWall,21.5,3.6,1.0,8.9,1.6,12.0
11,CPaul,15.8,4.3,0.3,8.2,2.2,15.0
Total,TeamRic,253.3,79.9,8.1,68.7,17.6,185.0


# E's team

In [89]:
#Dictionary of Ham's players so that the for loop can input the data into player_data_Ham dictionary 
#for all player stats

TeamElliott = {
    "KLowry": "http://data.nba.net/10s/prod/v1/2018/players/200768_profile.json",
    "Lonzo": "http://data.nba.net/10s/prod/v1/2018/players/1628366_profile.json",
    "EPayton": "http://data.nba.net/10s/prod/v1/2018/players/203901_profile.json",
    "Klay": "http://data.nba.net/10s/prod/v1/2018/players/202691_profile.json",
    "Draymond": "http://data.nba.net/10s/prod/v1/2018/players/203110_profile.json",
    "AD": "http://data.nba.net/10s/prod/v1/2018/players/203076_profile.json",
    "Embiid": "http://data.nba.net/10s/prod/v1/2018/players/203954_profile.json",
    "Millsap": "http://data.nba.net/10s/prod/v1/2018/players/200794_profile.json",
    "Capela": "http://data.nba.net/10s/prod/v1/2018/players/203991_profile.json",
    "DeMarcus": "http://data.nba.net/10s/prod/v1/2018/players/202326_profile.json",
    "Whiteside": "http://data.nba.net/10s/prod/v1/2018/players/202355_profile.json",
    "Rudy": "http://data.nba.net/10s/prod/v1/2018/players/200752_profile.json"  
    }

#Create empty Dictionary for Ham's players, player_data_Ham
player_data_Elliott = {}

#loop through the values ie the .json links and pull the stats for seasonYear, ppg,rpg,bpg,apg,spg and into into the
#empty dictionary of player_data_Ham

for key, val in TeamElliott.items():
    url = urllib.request.urlopen(val)
    response = json.loads(url.read().decode())
    df = response['league']['standard']['stats']['regularSeason']['season']
    df = json_normalize(df, 'teams')
    df = pd.DataFrame(df)
    df = pd.concat([dfseason, df], axis=1)
    df = df[['seasonYear','ppg','rpg','bpg','apg','spg','teamId']]
    df['name'] = key
    player_data_Elliott[key] = df


In [90]:
dfE1 = player_data_Elliott['KLowry']
dfE1 = pd.DataFrame(dfE1)
dfE1 = dfE1.iloc[[0]]

dfE2 = player_data_Elliott['Lonzo']
dfE2 = pd.DataFrame(dfE2)
dfE2 = dfE2.iloc[[0]]

dfE3 = player_data_Elliott['EPayton']
dfE3 = pd.DataFrame(dfE3)
dfE3 = dfE3.iloc[[0]]

dfE4 = player_data_Elliott['Klay']
dfE4 = pd.DataFrame(dfE4)
dfE4 = dfE4.iloc[[0]]

dfE5 = player_data_Elliott['Draymond']
dfE5 = pd.DataFrame(dfE5)
dfE5 = dfE5.iloc[[0]]

dfE6 = player_data_Elliott['AD']
dfE6 = pd.DataFrame(dfE6)
dfE6 = dfE6.iloc[[0]]

dfE7 = player_data_Elliott['Embiid']
dfE7 = pd.DataFrame(dfE7)
dfE7 = dfE7.iloc[[0]]

dfE8 = player_data_Elliott['Millsap']
dfE8 = pd.DataFrame(dfE8)
dfE8 = dfE8.iloc[[0]]

dfE9 = player_data_Elliott['Capela']
dfE9 = pd.DataFrame(dfE9)
dfE9 = dfE9.iloc[[0]]

dfE10 = player_data_Elliott['DeMarcus']
dfE10 = pd.DataFrame(dfE10)
dfE10 = dfE10.iloc[[0]]

dfE11 = player_data_Elliott['Whiteside']
dfE11 = pd.DataFrame(dfE11)
dfE11 = dfE11.iloc[[0]]

dfE12 = player_data_Elliott['Rudy']
dfE12 = pd.DataFrame(dfE12)
dfE12 = dfE12.iloc[[0]]

dfTEAMElliott = [dfE12, dfE11, dfE10, dfE9, dfE8, dfE7, dfE6, dfE5, dfE4, dfE3, dfE2, dfE1]
dfTEAMElliott = pd.concat(dfTEAMElliott)
#Add team wins category
dfTEAMElliott = dfTEAMElliott.merge(dfteam,on='teamId')

#change float

dfTEAMElliott['ppg'] = dfTEAMElliott['ppg'].astype(float)
dfTEAMElliott['rpg'] = dfTEAMElliott['rpg'].astype(float)
dfTEAMElliott['bpg'] = dfTEAMElliott['bpg'].astype(float)
dfTEAMElliott['apg'] = dfTEAMElliott['apg'].astype(float)
dfTEAMElliott['spg'] = dfTEAMElliott['spg'].astype(float)
dfTEAMElliott['win'] = dfTEAMElliott['win'].astype(float)


#dfTEAMElliott.dtypes


dfTEAMElliott.loc['Total']= dfTEAMElliott.sum()
dfTEAMElliott

dfTEAMElliott.iloc[-1, dfTEAMElliott.columns.get_loc('seasonYear')] = 2018
dfTEAMElliott.iloc[-1, dfTEAMElliott.columns.get_loc('name')] = 'TeamElliott'
dfTEAMElliott = dfTEAMElliott[['name','ppg', 'rpg','bpg','apg','spg','win']]


dfTEAMElliott.tail()



,name,ppg,rpg,bpg,apg,spg,win
8,Draymond,6.4,7.9,0.8,7.2,1.7,21.0
9,Klay,22.4,4.2,0.6,2.0,1.1,21.0
10,Lonzo,8.6,5.2,0.4,5.0,1.5,18.0
11,KLowry,14.2,4.4,0.4,10.0,1.4,23.0
Total,TeamElliott,199.4,104.3,15.4,49.7,13.8,212.0


# Sam's team

In [91]:
#Dictionary of Ham's players so that the for loop can input the data into player_data_Ham dictionary 
#for all player stats

TeamSam = {
    "JHarden": "http://data.nba.net/10s/prod/v1/2018/players/201935_profile.json",
    "Holiday": "http://data.nba.net/10s/prod/v1/2018/players/201950_profile.json",
    "Oladipo": "http://data.nba.net/10s/prod/v1/2018/players/203506_profile.json",
    "DMitchell": "http://data.nba.net/10s/prod/v1/2018/players/1628378_profile.json",
    "Simmons": "http://data.nba.net/10s/prod/v1/2018/players/1627732_profile.json",
    "Butler": "http://data.nba.net/10s/prod/v1/2018/players/202710_profile.json",
    "JaylenB": "http://data.nba.net/10s/prod/v1/2018/players/1627759_profile.json",
    "Middletown": "http://data.nba.net/10s/prod/v1/2018/players/203114_profile.json",
    "Drummond": "http://data.nba.net/10s/prod/v1/2018/players/203083_profile.json",
    "Jokic": "http://data.nba.net/10s/prod/v1/2018/players/203999_profile.json",
    "DSJunior": "http://data.nba.net/10s/prod/v1/2018/players/1628372_profile.json",
    "KrisDunn": "http://data.nba.net/10s/prod/v1/2018/players/1627739_profile.json"
    }

#Create empty Dictionary for Ham's players, player_data_Ham
player_data_Sam = {}

#loop through the values ie the .json links and pull the stats for seasonYear, ppg,rpg,bpg,apg,spg and into into the
#empty dictionary of player_data_Ham
for key, val in TeamSam.items():
    url = urllib.request.urlopen(val)
    response = json.loads(url.read().decode())
    df = response['league']['standard']['stats']['regularSeason']['season']
    df = json_normalize(df, 'teams')
    df = pd.DataFrame(df)
    df = pd.concat([dfseason, df], axis=1)
    df = df[['seasonYear','ppg','rpg','bpg','apg','spg','teamId']]
    df['name'] = key
    player_data_Sam[key] = df


In [92]:
dfS1 = player_data_Sam['JHarden']
dfS1 = pd.DataFrame(dfS1)
dfS1 = dfS1.iloc[[0]]

dfS2 = player_data_Sam['Holiday']
dfS2 = pd.DataFrame(dfS2)
dfS2 = dfS2.iloc[[0]]

dfS3 = player_data_Sam['Oladipo']
dfS3 = pd.DataFrame(dfS3)
dfS3 = dfS3.iloc[[0]]

dfS4 = player_data_Sam['DMitchell']
dfS4 = pd.DataFrame(dfS4)
dfS4 = dfS4.iloc[[0]]

dfS5 = player_data_Sam['Simmons']
dfS5 = pd.DataFrame(dfS5)
dfS5 = dfS5.iloc[[0]]

dfS6 = player_data_Sam['Butler']
dfS6 = pd.DataFrame(dfS6)
dfS6 = dfS6.iloc[[0]]

dfS7 = player_data_Sam['JaylenB']
dfS7 = pd.DataFrame(dfS7)
dfS7 = dfS7.iloc[[0]]

dfS8 = player_data_Sam['Middletown']
dfS8 = pd.DataFrame(dfS8)
dfS8 = dfS8.iloc[[0]]

dfS9 = player_data_Sam['Drummond']
dfS9 = pd.DataFrame(dfS9)
dfS9 = dfS9.iloc[[0]]

dfS10 = player_data_Sam['Jokic']
dfS10 = pd.DataFrame(dfS10)
dfS10 = dfS10.iloc[[0]]

dfS11 = player_data_Sam['DSJunior']
dfS11 = pd.DataFrame(dfS11)
dfS11 = dfS11.iloc[[0]]

dfS12 = player_data_Sam['KrisDunn']
dfS12 = pd.DataFrame(dfS12)
dfS12 = dfS12.iloc[[0]]

dfTEAMSAM = [dfS12, dfS11, dfS10, dfS9, dfS8, dfS7, dfS6, dfS5, dfS4, dfS3, dfS2, dfS1]
dfTEAMSAM = pd.concat(dfTEAMSAM)

#Add team wins category
dfTEAMSAM = dfTEAMSAM.merge(dfteam,on='teamId')

#change float

dfTEAMSAM['ppg'] = dfTEAMSAM['ppg'].astype(float)
dfTEAMSAM['rpg'] = dfTEAMSAM['rpg'].astype(float)
dfTEAMSAM['bpg'] = dfTEAMSAM['bpg'].astype(float)
dfTEAMSAM['apg'] = dfTEAMSAM['apg'].astype(float)
dfTEAMSAM['spg'] = dfTEAMSAM['spg'].astype(float)
dfTEAMSAM['win'] = dfTEAMSAM['win'].astype(float)

#dfTEAMSAM.dtypes



dfTEAMSAM.loc['Total']= dfTEAMSAM.sum()
dfTEAMSAM.iloc[-1, dfTEAMSAM.columns.get_loc('seasonYear')] = 2018
dfTEAMSAM.iloc[-1, dfTEAMSAM.columns.get_loc('name')] = 'TeamSam'
dfTEAMSAM = dfTEAMSAM[['name','ppg', 'rpg','bpg','apg','spg','win']]


dfTEAMSAM



,name,ppg,rpg,bpg,apg,spg,win
0,KrisDunn,11.8,4.0,0.0,5.0,0.8,7.0
1,DSJunior,13.0,3.0,0.5,4.0,1.3,15.0
2,Jokic,17.7,9.9,0.7,7.5,1.4,20.0
3,Drummond,18.0,15.5,1.9,1.2,1.4,14.0
4,Middletown,17.6,5.8,0.1,3.9,1.2,20.0
5,JaylenB,11.7,4.2,0.6,1.5,0.7,18.0
6,Butler,18.8,4.8,0.6,2.8,1.7,20.0
7,Simmons,15.9,9.0,0.8,7.9,1.4,20.0
8,DMitchell,21.0,3.1,0.3,3.2,1.7,14.0
9,Oladipo,20.8,6.6,0.4,5.1,1.7,20.0


# League totals

In [93]:
#Create league totals into a dataframe
dfleague = [dfTEAMHam, dfTEAMElliott, dfTEAMRic, dfTEAMSAM]
dfleague = pd.concat(dfleague)
dfleague.reset_index()
dfleague = dfleague.loc[ 'Total' , : ]
dfleague = dfleague.loc[:, ['name','ppg', 'rpg','bpg','apg','spg','win']]

#Add rank columns for all categories
dfleague['ppgR'] = dfleague['ppg'].rank(ascending=False)
dfleague['rpgR'] = dfleague['rpg'].rank(ascending=False)
dfleague['bpgR'] = dfleague['bpg'].rank(ascending=False)
dfleague['apgR'] = dfleague['apg'].rank(ascending=False)
dfleague['spgR'] = dfleague['spg'].rank(ascending=False)
dfleague['winR'] = dfleague['win'].rank(ascending=False)

#Add average of the categories
dfleague['Overall'] = dfleague[['ppgR','rpgR','bpgR','apgR','spgR','winR']].mean(axis=1)
dfleague.round(2)


,name,ppg,rpg,bpg,apg,spg,win,ppgR,rpgR,bpgR,apgR,spgR,winR,Overall
Total,TeamHam,272.1,93.9,9.0,61.4,12.1,193.0,1.0,2.0,2.0,2.0,4.0,3.0,2.33
Total,TeamElliott,199.4,104.3,15.4,49.7,13.8,212.0,4.0,1.0,1.0,4.0,3.0,1.0,2.33
Total,TeamRic,253.3,79.9,8.1,68.7,17.6,185.0,2.0,3.0,3.0,1.0,1.0,4.0,2.33
Total,TeamSam,218.3,76.0,7.1,59.0,17.3,198.0,3.0,4.0,4.0,3.0,2.0,2.0,3.00


# Email the data

In [94]:
# TESTING
#This code works:

sender = "xyz@gmail.com"
#unhashtag below to get email recipents to send
recipients = ["ricjfernandez@gmail.com","samyoung15@gmail.com","elliott.char@gmail.com","hamungpatel@hotmail.com","siningzhou@gmail.com"]
msg = MIMEMultipart('mixed') #use mixed instead of alternative to load multiple things 
msg['Subject'] = "The League 2018-2019 Official Data Source"
msg['From'] = sender
msg['To'] = ", ".join(recipients)

#first dataframe
html = dfleague.to_html() 
#insert text as follows

html += '''
    <br><br>
    All data is sourced from the NBA. Below are the individual team stats per person.
    <br><br>
'''
#second and so on dataframes
html += '''
    <br><br>
    Team Ham
    <br><br>
'''
html += dfTEAMHam.to_html()

html += '''
    <br><br>
    Team E
    <br><br>
'''

html += dfTEAMElliott.to_html() 

html += '''
    <br><br>
    Team Ric
    <br><br>
'''



html += dfTEAMRic.to_html() 

html += '''
    <br><br>
    Team Sam
    <br><br>
'''



html += dfTEAMSAM.to_html() 




#put the htmls into the email body
html = MIMEText(html, 'html') 
msg.attach(html)

#send the email out
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login("xyz@gmail.com", "password")
server.sendmail(sender, recipients, msg.as_string())
server.quit()  






(221, b'2.0.0 closing connection h129sm4658426wma.0 - gsmtp')